**Outline:**
- Impute Time
    + Dummy 1 to impute Time
    + Dummy 2 to impute Time
- Impute Activity

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
from math import sqrt

In [2]:
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, log_loss

In [3]:
sys.path.insert(0, './../utils/')
from utils import *

In [4]:
#Define parser
#name = 'bpi_2012'
name = 'bpi_2013'
#name = 'Road_Traffic_Fine_Management_Process'  

parser = {
    'data_file': name + '.csv',
    'nan_pct': 0.3,
    'input_dir': '../input/{}/'.format(name), 
    'output_dir': './output/{}/'.format(name),
}

args = argparse.Namespace(**parser)

In [5]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

In [20]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [21]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [24]:
complete_true_test['CompleteTimestamp'] = pd.to_datetime(complete_true_test['CompleteTimestamp'], errors='coerce')

In [25]:
complete_true_test.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,1190,Accepted-In Progress,2012-03-07 22:24:22,194135680.0,0.0
1,1190,Completed-Closed,2012-04-11 21:36:11,197156789.0,1.0
2,1191,Accepted-In Progress,2012-03-07 22:26:08,194135786.0,0.0
3,1191,Completed-Closed,2012-04-25 16:42:09,198348747.0,1.0
4,1192,Accepted-In Progress,2012-03-08 00:06:16,194141794.0,0.0


In [26]:
groupByCase = complete_true_test.groupby(['CaseID'])

In [30]:
duration_list = []
for caseid, data_case in groupByCase:
    for row in range(data_case.shape[0]):
        if row == 0:
            duration_case = 0
        else:
            current_time = data_case.iloc[row, data_case.columns.get_loc('CompleteTimestamp')]
            previous_time = data_case.iloc[row - 1, data_case.columns.get_loc('CompleteTimestamp')]
            duration_case = (current_time - previous_time).total_seconds()
        duration_list.append(duration_case)

In [ ]:
len(duration_list) == complete_true_test.shape[0]

In [ ]:
complete_true_test['Duration'] = duration_list

In [37]:
total_duration_list = []
for caseid, data_case in groupByCase:
    last_time = data_case.iloc[-1, data_case.columns.get_loc('CompleteTimestamp')]
    first_time = data_case.iloc[0, data_case.columns.get_loc('CompleteTimestamp')]
    total_duration_case = (last_time - first_time).total_seconds()
    total_duration_list.append(total_duration_case)

In [38]:
len(total_duration_list) == len(groupByCase)

True

In [41]:
np.mean(total_duration_list)/86400

16.301325775229927

In [42]:
np.min(total_duration_list)/86400

0.0

In [43]:
np.max(total_duration_list)/86400

75.15234953703704

In [44]:
np.median(total_duration_list)/86400

8.9238599537037029

In [35]:
complete_true_test.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime,Duration
0,1190,Accepted-In Progress,2012-03-07 22:24:22,194135680.0,0.0,0.0
1,1190,Completed-Closed,2012-04-11 21:36:11,197156789.0,1.0,3021109.0
2,1191,Accepted-In Progress,2012-03-07 22:26:08,194135786.0,0.0,0.0
3,1191,Completed-Closed,2012-04-25 16:42:09,198348747.0,1.0,4212961.0
4,1192,Accepted-In Progress,2012-03-08 00:06:16,194141794.0,0.0,0.0


In [36]:
complete_true_test['Duration'].describe()

count    1.071000e+03
mean     3.918894e+05
std      9.358506e+05
min      0.000000e+00
25%      0.000000e+00
50%      3.000000e+02
75%      1.656090e+05
max      6.493163e+06
Name: Duration, dtype: float64